# Designing Specifications

Objective

* 了解非确定性规范，并能识别和评估非确定性规范
* Understand declarative vs. operational specs, and be able to write declarative specs
* 了解前提条件、后置条件和规范的强度，并能比较规范的强度
* 能够编写连贯、有用、强度适当的规范


## Introduction
In this reading we’ll look at different specs for similar behaviors, and talk about the tradeoffs between them. We’ll look at three dimensions for comparing specs:

* How deterministic it is. Does the spec define only a single possible output for a given input, or does it permit a set of legal outputs?

* How declarative it is. Does the spec just characterize what the output should be, or does it explicitly say how to compute the output?

* How strong it is. Does the spec have a small set of legal implementations, or a large set?

* Not all specifications we might choose for a module are equally useful, and we’ll explore what makes some specifications better than others.

## Deterministic vs. underdetermined specs
```TS
function findFirst(arr: Array<number>, val: number): number {
    for (let i = 0; i < arr.length; i++) {
        if (arr[i] === val) return i;
    }
    return arr.length;
}

function findLast(arr: Array<number>, val: number): number {
    for (let i = arr.length - 1 ; i >= 0; i--) {
        if (arr[i] === val) return i;
    }
    return -1;
}
```

下标 First 和 Last 并不是真正的 TypeScript 语法。为了便于讨论，我们在此使用它们来区分两种实现。在实际代码中，这两种实现都应该是名为 find 的 TypeScript 方法。

因为我们也会讨论 find 的多种规范，所以我们会用一个上标来标识每种规范，比如 ExactlyOne：

```
function findExactlyOne(arr: Array<number>, val: number): number
requires: val occurs exactly once in arr
effects: returns index i such that arr[i] = val
```

findExactlyOne 规范是deterministic：当出现满足先决条件的状态时，结果是完全确定的。只有一个返回值和一个最终状态是可能的。没有任何有效输入会有一个以上的有效输出。

findFirst 和 findLast 的实现都满足该规范的要求，因此，如果这是客户所依赖的规范，那么这两个实现是可以相互替代的。

下面是一个稍有不同的规范：

```
function findOneOrMore,AnyIndex(arr: Array<number>, val: number): number
requires: val occurs in arr
effects: returns index i such that arr[i] = val
```
该规范不是deterministic。它没有说明如果 val 出现多次，会返回哪个索引。它只是说，如果按照返回值给出的索引查找条目，就能找到 val。在某些输入上，该规范允许多个有效输出。

尽管这种规范不是deterministic，但我们不会把它称为通常意义上的nondeterministic。Nondeterministic代码有时表现为一种方式，有时又表现为另一种方式，即使在同一程序中以相同的输入调用也是如此。例如，当代码的行为取决于随机数，或取决于并发进程的时序时，就会出现这种情况。但是，非确定性规范并不一定需要非确定性实现。完全确定的实现也能满足它。

为了避免混淆，我们把not deterministic spec 称为underdetermined spec。

underdetermined findOneOrMore,AnyIndex spec同样可以通过 findFirst 和 findLast 来实现，它们各自以自己的（fully deterministic）方式解决underdeterminedness 问题。如果 val 出现不止一次，该规范的客户端就不能依赖于返回哪个索引。非确定性实现也能满足该规范的要求，例如，可以通过掷硬币来决定从数组的开头还是结尾开始搜索。但在我们会遇到的几乎所有情况下，规范中的非确定性都提供了一种由实现者在实现时做出的选择。未完全确定的规范通常由完全确定的实现来实现。
findFirst 和 findLast 都能满足这种查找一个或多个、任意索引的欠确定性规范，它们各自以自己的（完全确定的）方式解决欠确定性问题。如果 val 出现不止一次，该规范的客户端就不能依赖于返回哪个索引。非确定性实现也能满足该规范的要求，例如，可以通过掷硬币来决定从数组的开头还是结尾开始搜索。但在我们会遇到的几乎所有情况下，规范中的非确定性都提供了一种由实现者在实现时做出的选择。未完全确定的规范通常由完全确定的实现来实现。

## Declarative vs. operational specs
在这个比较维度上，有两种规范。操作性规范给出了方法执行的一系列步骤；伪代码描述是操作性的。声明性规范不提供中间步骤的细节。相反，它们只给出最终结果的属性，以及它与初始状态的关系。

声明式规范几乎总是优于操作式规范。它们通常更简短、更容易理解，而且最重要的是，它们不会无意中暴露客户可能依赖的实现细节（当实现改变时，客户会发现这些细节不再有效）。例如，如果我们想允许 find 的任何一种实现，我们就不想在规范中说明该方法 "沿着数组向下搜索，直到找到 val"，因为除了相当含糊之外，该规范还暗示搜索将从低索引向高索引进行，并将返回最低的索引，而这也许并不是规范制定者的本意。

程序员有时会陷入操作规范的误区，原因之一是他们使用规范注释为维护者解释实现。不要这样做。必要时，应在方法正文中使用注释，而不是在规范注释中。

对于给定的规范，可能有很多方法可以用声明的方式来表达：

```
function startsWith(str: string, prefix: string): boolean
effects: returns true if and only if there exists a string suffix such that prefix + suffix = str

function startsWith(str: string, prefix: string): boolean
effects: returns true if and only if there exists integer i such that str.substring(0, i) = prefix

function startsWith(str: string, prefix: string): boolean
effects: returns true if and only if the first prefix.length characters of str are the characters of prefix
```
我们可以为客户端和代码维护者选择最清晰的规范。

请注意，这些 startsWith 规范没有先决条件，因此我们可以省略 requirements： nothing for the sake of brevity.

## Stronger vs. weaker specs

我们在上一篇文章（specition）中讨论过，只要两个实现都符合规范，那么规范的存在就可以让你安全地用另一个实现来替换一个实现。但假设你不仅需要改变实现，还需要改变规范本身呢？假设已经有客户依赖于方法的当前规范。如何比较两种规范的行为，以决定用新规范替换旧规范是否安全（对这些客户而言）？

要回答这个问题，我们需要比较两种规范的强度。如果满足 S2 的实现集是满足 S1 的实现集的严格子集，那么规范 S2 就比规范 S1 强。

强 "和 "弱 "的概念来自谓词逻辑 [predicate logic](https://en.wikipedia.org/wiki/First-order_logic)。如果符合 P 的状态集是符合 Q 的状态集的严格子集，那么谓词 P 就比谓词 Q 强（Q 比 P 弱）。把 "更强 "理解为更严格的限制，而把 "更弱 "理解为更宽松的限制。

请注意，如果谓词 P 和 Q 都不是另一个谓词的子集，那么这两个谓词也有可能是不可比的--既不比另一个谓词强，也不比另一个谓词弱。

要判断一个规范比另一个规范强或弱，我们要比较它们的前置条件和后置条件的强度。先决条件是对输入状态的谓词，因此增强先决条件意味着缩小合法输入的集合。同样，后置条件是对输出状态的谓词，因此一个更强的后置条件会缩小允许输出和效果的集合。

现在我们可以陈述规则了：
```
一个规范 S2 强于或等于一个规范 S1，当且仅当

S2 的前提条件弱于或等于 S1 的前提条件、
且
对于满足 S1 前提条件的状态，S2 的后置条件强于或等于 S1 的后置条件。
如果是这种情况，那么满足 S2 的实现也可以用来满足 S1，而且用 S2 代替 S1 对客户来说也是安全的。
```

这条规则体现了几个理念。它告诉我们，我们可以在不影响客户的情况下弱化先决条件，因为对客户提出更少的要求永远不会让他们感到不安--弱化后的先决条件仍能满足他们的输入，而且还允许更多以前不合法的输入。
你还可以加强后置条件，这意味着向客户做出更多承诺--新的后置条件允许的输出可能比以前少，但它将是客户已经准备好处理的输出的子集。

For example, this spec for find:
```
function findExactlyOne(a: Array<number>, val: number): number
requires: val occurs exactly once in a
effects: returns index i such that a[i] = val

```
can be replaced with:
```
function findOneOrMore,AnyIndex(a: Array<number>, val: number): number
requires: val occurs at least once in a
effects: returns index i such that a[i] = val
```
which is a stronger spec because it has a weaker precondition — it constrains the inputs less. This in turn can be replaced with:
```
function findOneOrMore,FirstIndex(a: Array<number>, val: number): number
requires: val occurs at least once in a
effects: returns lowest index i such that a[i] = val
```
which is a stronger spec because it has a stronger postcondition — it constrains the output more.

What about this specification:

```
function findCanBeMissing(a: Array<number>, val: number): number
requires: nothing
effects: returns index i such that a[i] = val, or -1 if no such i

```
让我们与 findOneOrMore,FirstIndex 进行比较。同样，前提条件更弱，但对于满足 findOneOrMore,FirstIndex 前提条件的输入，后置条件也更弱：取消了对最低索引的要求。这两种规范都不比另一种强：它们是不可比的。

我们将在练习中再次讨论 findCanBeMissing，并将它与其他规范进行比较。